----
<img src="./files/refinitiv.png" width="20%" style="vertical-align: top;">

----
    This source code is provided under the Apache 2.0 license
    and is provided AS IS with no warranty or guarantee of fit for purpose.
    Copyright (C) 2022 Refinitiv. All rights reserved.
----

# Requesting 2K Pricing Streaming Events with Refinitiv Data Library

This example code is based on EX-2.02.04-Pricing-StreamingEvents example, that is part of Refinitiv Data Library for Python example deck


In [29]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [30]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
import datetime
import pandas as pd

In [31]:
# optional for Debugging
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)

In [32]:
# optional for displaying
pd.set_option('display.max_rows', None)

###  Read File with RICs

In [33]:
defaultRICs =  ['IBM.N', 'GBP=', 'JPY=', 'CAD=']  
def readRICFile(filename):
    if filename != '':
        f = open(filename, "r")
        ricList = f.read().splitlines()
        print('Read '+str(len(ricList))+' RICs from file '+filename)
    else:
        ricList = defaultRICs
    return ricList
#myRics = readRICFile('./nasd100.sf.txt')
myRics = readRICFile('Rics2K.txt')

Read 2000 RICs from file Rics2K.txt


### Open the data session

Supported sesstions are:
*  Desktop
*  RDP platform human or machineId if permissioned for streaming pricing
*  Deployed RTDS

In [34]:
#rd.open_session()
#rd.open_session('platform.rdpMachine')
rd.open_session(name='platform.rdpMachine',config_name='../../../Configuration/refinitiv-data.config.json')
#rd.open_session(name='platform.deployed',config_name='../../../Configuration/refinitiv-data.config.json')
#rd.open_session(name='desktop.workspace',config_name='../../../Configuration/refinitiv-data.config.json')
#rd.open_session(name='platform.rdp',config_name='../../../Configuration/refinitiv-data.config.json')

2022-06-02 18:27:50,975 - httpx._client - DEBUG - HTTP Request: POST https://api.refinitiv.com/auth/oauth2/v1/token "HTTP/1.1 200 OK"
2022-06-02 18:27:51,006 - charset_normalizer - INFO - ascii passed initial chaos probing. Mean measured chaos is 0.000000 %
2022-06-02 18:27:51,007 - charset_normalizer - INFO - ascii should target any language(s) of ['Latin Based']
2022-06-02 18:27:51,013 - charset_normalizer - INFO - We detected language [('English', 0.9923), ('Simple English', 0.9923), ('Indonesian', 0.9923)] using ascii
2022-06-02 18:27:51,013 - charset_normalizer - INFO - ascii is most likely the one. Stopping the process.


<refinitiv.data.session.platform.Definition object at 0x1fb1e2c7760 {session_name='rdpMachine'}>

## Request Streaming Data

### Define callbacks to capture incoming events

In [35]:
def display_refreshed_fields(pricing_stream, instrument_name, fields):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Refresh received for", instrument_name, ":", fields)    

In [36]:
def display_updated_fields(pricing_stream, instrument_name, fields):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Update received for", instrument_name, ":", fields)    

In [37]:
def display_status(pricing_stream, instrument_name, status):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Status received for", instrument_name, ":", status)    

In [38]:
def display_complete_snapshot(pricing_stream):
    current_time = datetime.datetime.now().time()
    print(current_time, "- Pricing stream is complete. Full snapshot:")
    display(pricing_stream.get_snapshot())

### Create a Pricing stream and register event callbacks

In [39]:
stream = rd.content.pricing.Definition(
    myRics,
#    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()


stream.on_refresh(display_refreshed_fields)
stream.on_update(display_updated_fields)
stream.on_status(display_status)
stream.on_complete(display_complete_snapshot)

<refinitiv.data.content.pricing.Stream object at 0x1fb1e2c78b0 {name='['VALE3.SA', '6454.T', 'NATS.SI', 'ADP.OQ', '6457.T', '9720.T', '8095.T', '1914.T', '1992.T', '7953.T', '6709.T', '4362.T', '1980.T', '5939.T', 'MSED.PA', 'ZDYu.TO', '2651.T', 'ERIE.OQ', '7914.T', 'CTO.L', '6168.TW', 'BAFS.BK', 'CITT.PA', 'TESS.OQ', 'GHE.L', 'HLMA.L', 'LAWS.OQ', '6592.T', 'NSSC.OQ', 'TGB.A', '1878.T', '9001.T', '8051.T', '3HCL.L', '159918.SZ', 'MAHB.KL', '9856.T', 'CHSS.PA', 'MGIC.OQ', 'PBI.N', 'DWHT.L', 'ERC.A', 'VERB.F', '8518.MU', 'MMM.N', 'XSB.TO', '4215.T', '7250.T', '9082.T', '5191.T', '9631.T', '9945.T', '013360.KS', '004250.KS', 'IFRB.L', '006206.TW', 'AMGN.OQ', 'CLII.MI', 'AGG.AX', '4745.T', 'STRS.TA', 'FTF.A', 'RCG.A', '0051.TW', '2821.HK', 'IONAF.PK', '2811.T', 'SPYJ.F', 'REMJ.J', '8115.T', '2305.TW', '000680.SZ', 'FEN.A', '8035.BE', 'OEC.F', '4662.BE', '3019.TW', 'SKBC.KL', 'IOMT.PK', 'ZPRL.F', 'JBHT.OQ', 'OERL.S', '5363.T', 'EGHSF.PK', '7769.T', '3577.T', '8074.T', 'TURY.KL', 'KWEB.P', '

### Create a Pricing stream for Deployed session- may need to set service explicitely 
If the service name does not match, the error may look similar to:

...'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'SourceUnknown', 'Text': 'A18: Unknown service.'...

In [12]:
stream = rd.content.pricing.Definition(
    myRics,
#    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK'],
#    service = 'ELEKTRON_DD'
).get_stream()


stream.on_refresh(display_refreshed_fields)
stream.on_update(display_updated_fields)
stream.on_status(display_status)
stream.on_complete(display_complete_snapshot)

<refinitiv.data.content.pricing.Stream object at 0x1d90d818bb0 {name='['VALE3.SA', '6454.T', 'NATS.SI', 'ADP.OQ', '6457.T', '9720.T', '8095.T', '1914.T', '1992.T', '7953.T', '6709.T', '4362.T', '1980.T', '5939.T', 'MSED.PA', 'ZDYu.TO', '2651.T', 'ERIE.OQ', '7914.T', 'CTO.L', '6168.TW', 'BAFS.BK', 'CITT.PA', 'TESS.OQ', 'GHE.L', 'HLMA.L', 'LAWS.OQ', '6592.T', 'NSSC.OQ', 'TGB.A', '1878.T', '9001.T', '8051.T', '3HCL.L', '159918.SZ', 'MAHB.KL', '9856.T', 'CHSS.PA', 'MGIC.OQ', 'PBI.N', 'DWHT.L', 'ERC.A', 'VERB.F', '8518.MU', 'MMM.N', 'XSB.TO', '4215.T', '7250.T', '9082.T', '5191.T', '9631.T', '9945.T', '013360.KS', '004250.KS', 'IFRB.L', '006206.TW', 'AMGN.OQ', 'CLII.MI', 'AGG.AX', '4745.T', 'STRS.TA', 'FTF.A', 'RCG.A', '0051.TW', '2821.HK', 'IONAF.PK', '2811.T', 'SPYJ.F', 'REMJ.J', '8115.T', '2305.TW', '000680.SZ', 'FEN.A', '8035.BE', 'OEC.F', '4662.BE', '3019.TW', 'SKBC.KL', 'IOMT.PK', 'ZPRL.F', 'JBHT.OQ', 'OERL.S', '5363.T', 'EGHSF.PK', '7769.T', '3577.T', '8074.T', 'TURY.KL', 'KWEB.P', '

### Open the stream

In [40]:
stream.open()

2022-06-02 18:28:24,340 - httpx._client - DEBUG - HTTP Request: GET https://api.refinitiv.com/streaming/pricing/v1/ "HTTP/1.1 200 OK"
2022-06-02 18:28:24,342 - charset_normalizer - INFO - ascii passed initial chaos probing. Mean measured chaos is 0.000000 %
2022-06-02 18:28:24,344 - charset_normalizer - INFO - ascii should target any language(s) of ['Latin Based']
2022-06-02 18:28:24,345 - charset_normalizer - INFO - We detected language [('English', 1.0), ('Dutch', 1.0), ('German', 0.9714)] using ascii
2022-06-02 18:28:24,347 - charset_normalizer - INFO - ascii is most likely the one. Stopping the process.


18:28:32.020526 - Refresh received for VALE3.SA : {'BID': 89.89, 'ASK': 89.91}
18:28:32.382524 - Refresh received for 6454.T : {'BID': None, 'ASK': None}
18:28:32.383524 - Refresh received for ADP.OQ : {'BID': 223.37, 'ASK': 224.73}
18:28:32.384524 - Refresh received for NATS.SI : {'BID': None, 'ASK': None}
18:28:32.387524 - Refresh received for 6457.T : {'BID': None, 'ASK': None}
18:28:32.387524 - Refresh received for 9720.T : {'BID': None, 'ASK': None}
18:28:32.388523 - Refresh received for 8095.T : {'BID': None, 'ASK': None}
18:28:32.391525 - Refresh received for 1914.T : {'BID': None, 'ASK': None}
18:28:32.395524 - Refresh received for 1992.T : {'BID': None, 'ASK': None}
18:28:32.397524 - Refresh received for 7953.T : {'BID': None, 'ASK': None}
18:28:32.400524 - Refresh received for 1980.T : {'BID': None, 'ASK': None}
18:28:32.401524 - Refresh received for 5939.T : {'BID': None, 'ASK': None}
18:28:32.401524 - Refresh received for 4362.T : {'BID': None, 'ASK': None}
18:28:32.402524 

,Instrument,BID,ASK
0,VALE3.SA,89.89,89.91
1,6454.T,<NA>,<NA>
2,NATS.SI,<NA>,<NA>
3,ADP.OQ,223.37,224.73
4,6457.T,<NA>,<NA>
5,9720.T,<NA>,<NA>
6,8095.T,<NA>,<NA>
7,1914.T,<NA>,<NA>
8,1992.T,<NA>,<NA>
9,7953.T,<NA>,<NA>


18:28:34.017821 - Status received for POCEF.PK : {'ID': 33777, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'POCEF.PK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:28:34.020857 - Update received for AAPL.OQ : {'BID': 151.1, 'ASK': 151.4}


<OpenState.Opened: 1>

18:28:37.992576 - Update received for EA.OQ : {'BID': 140.31, 'ASK': 142.09}
18:28:38.527977 - Update received for AAPL.OQ : {'BID': 151.1, 'ASK': 151.38}
18:28:38.813500 - Update received for PSQ.P : {'BID': 12.96, 'ASK': 12.97}
18:28:38.992502 - Update received for EA.OQ : {'BID': 141.13, 'ASK': 142.09}
18:28:39.430505 - Update received for QLD.P : {'BID': 52.86, 'ASK': 52.89}
18:28:39.804499 - Update received for IEMG.P : {'BID': 53.1, 'ASK': 53.46}
18:28:39.823504 - Update received for PSQ.P : {'BID': 12.96, 'ASK': 12.97}
18:28:39.967822 - Update received for VGK.P : {'BID': 60.1, 'ASK': 61.86}
18:28:40.318853 - Update received for AAPL.OQ : {'BID': 151.1, 'ASK': 151.34}
18:28:40.441890 - Update received for QLD.P : {'BID': 52.86, 'ASK': 52.88}
18:28:40.825431 - Update received for PSQ.P : {'BID': 12.96, 'ASK': 12.97}
18:28:41.290548 - Update received for AAPL.OQ : {'BID': 151.1, 'ASK': 151.34}
18:28:41.432943 - Update received for QLD.P : {'BID': 52.85, 'ASK': 52.88}
18:28:42.2830

### Close the stream (at any time)

In [41]:
stream.close()
#rd.close_session()

<OpenState.Closed: 3>

18:29:50.769882 - Update received for QLD.P : {'BID': 52.86, 'ASK': 52.88}


## Request Larger List of Streaming Data
Partitioning requests into convenient bathes

### Read 10k RIC List

In [54]:
my10kRics = readRICFile('Rics10K.txt')

Read 10000 RICs from file Rics10K.txt


### Chunk RIC List - Define a Helper Function

In [43]:
def list_to_chunks(long_list, chunk_size):
    chunked_list = list()
    for i in range(0, len(long_list), chunk_size):
        chunked_list.append(long_list[i:i+chunk_size])
    return chunked_list
#CHUNK_SIZE = 100
#myRicChunks = list_to_chunks(myRics, CHUNK_SIZE)
#print('My chunks are ',myRicChunks)

### Partion the List into 2k Sub-Lists (Batches)

In [55]:
my2kRicChunks = list_to_chunks(my10kRics, 2000)
#print('My chunks are ',my2kRicChunks)

### Create a Stream for Each Batch (Sub-List) with Callbacks
Minimize the output to be able to see the output reporting completion of each Batch - Refresh callbacks are just "pass"

In [48]:
def donot_display_refreshed_fields(pricing_stream, instrument_name, fields):
    pass
      
#def donot_display_updated_fields(pricing_stream, instrument_name, fields):
#    pass


streams_list = list()
for i in range(len(my2kRicChunks)):
    stream = rd.content.pricing.Definition(
        my2kRicChunks[i],
        fields=['BID', 'ASK']
    ).get_stream()

    stream.on_refresh(donot_display_refreshed_fields)
    stream.on_update(display_updated_fields)
    stream.on_status(display_status)
    stream.on_complete(display_complete_snapshot)
    
    streams_list.append(stream)
  

### Open Streams Sequentially

In [49]:
for i in range(len(streams_list)):
    streams_list[i].open()
    print("Stream #", i)

18:40:44.530727 - Status received for UWCmn.BK : {'ID': 43860, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'UWCmn.BK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:44.539743 - Status received for DIPGY.PK : {'ID': 44055, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'DIPGY.PK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:44.540728 - Status received for SRIK.BO : {'ID': 44039, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'SRIK.BO'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:44.571727 - Status received for GGM.N : {'ID': 43994, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'GGM.N'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:44.581

,Instrument,BID,ASK
0,PEK.AX,0.45,0.45
1,EMKA.BB,3.72,3.96
2,MIC.HNO,<NA>,12200.0
3,PROF.AM,0.56,0.58
4,3011.TW,17.6,17.65
5,DITAS.IS,<NA>,<NA>
6,GMG.AX,21.11,19.0
7,TAPa.N,0.0,0.0
8,ABBV.N,0.0,0.0
9,88S1.F,<NA>,<NA>


18:40:45.503744 - Status received for GAZPq.L : {'ID': 45802, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'GAZPq.L'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
Stream # 0
18:40:45.581777 - Update received for IVOO.P : {'BID': 172.36, 'ASK': 172.89}
18:40:47.547741 - Status received for AMBA.OQ : {'ID': 45801, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'AMBA.OQ'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'AlreadyOpen', 'Text': 'A46: Item was reopened under new stream.'}}
18:40:47.629775 - Status received for 0999.HK : {'ID': 45921, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': '0999.HK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:47.632775 - Status received for 2823.TW : {'ID': 45872, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': '2823.TW'}, 'State': {'Stream': 'Clo

,Instrument,BID,ASK
0,EEMS.P,40.0,0.0
1,1628.T,<NA>,<NA>
2,NOCI.BO,<NA>,<NA>
3,0022.HK,0.095,0.098
4,AJL.AX,0.056,0.06
5,4282.T,<NA>,<NA>
6,161000.KS,<NA>,<NA>
7,SKFb.ST,<NA>,<NA>
8,AFR.AX,<NA>,<NA>
9,2454.TW,898.0,899.0


18:40:48.361721 - Status received for WLTW.OQ : {'ID': 47794, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'WLTW.OQ'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
Stream # 1
18:40:49.568723 - Update received for URTY.P : {'BID': 55.34, 'ASK': 55.45}
18:40:49.568723 - Update received for VTWO.OQ : {'BID': 76.15, 'ASK': 76.26}
18:40:51.277768 - Update received for NVDA.OQ : {'BID': 197.24, 'ASK': 197.63}
18:40:51.353776 - Status received for KEN.N : {'ID': 47476, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'KEN.N'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'AlreadyOpen', 'Text': 'A46: Item was reopened under new stream.'}}
18:40:51.401782 - Status received for ILK1S.HE : {'ID': 47914, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'ILK1S.HE'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:

...

18:40:52.217742 - Status received for 4EM.BE : {'ID': 49814, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': '4EM.BE'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
Stream # 2
18:40:54.894775 - Status received for PUKPF.PK : {'ID': 49074, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'PUKPF.PK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'AlreadyOpen', 'Text': 'A46: Item was reopened under new stream.'}}
18:40:54.897741 - Status received for PLHNF.PK : {'ID': 49083, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'PLHNF.PK'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'AlreadyOpen', 'Text': 'A46: Item was reopened under new stream.'}}
18:40:54.951741 - Status received for LBTYA.OQ : {'ID': 49553, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'LBTYA.OQ'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'AlreadyOpen', 'Text': 'A46: 

,Instrument,BID,ASK
0,CNRAF.PK,<NA>,<NA>
1,ACSL.BO,<NA>,<NA>
2,ALIG.ST,<NA>,<NA>
3,1363.HK,<NA>,<NA>
4,3207.TWO,33.45,33.5
5,AAOI.OQ,2.0,0.0
6,CHLt.CM,95.0,104.5
7,ROUNVR.BX,<NA>,<NA>
8,RUMI.AM,<NA>,1.13
9,1296.HK,<NA>,<NA>


18:40:55.640655 - Status received for TACO.OQ : {'ID': 51737, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'TACO.OQ'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
18:40:55.640655 - Update received for SCO.P : {'BID': 20.15, 'ASK': 20.21}
18:40:55.640655 - Update received for XBI.P : {'BID': 68.79, 'ASK': 68.88}
18:40:55.640655 - Update received for XBI.P : {'BID': 68.79, 'ASK': 68.89}
18:40:55.641652 - Update received for TNA.P : {'BID': 46.33, 'ASK': 46.35}
18:40:55.641652 - Update received for UPRO.P : {'BID': 48.08, 'ASK': 48.11}
Stream # 3
18:40:55.733686 - Update received for SCO.P : {'BID': 20.16, 'ASK': 20.21}
18:40:55.931689 - Update received for IVV.P : {'BID': 419.75, 'ASK': 419.85}
18:40:56.110671 - Update received for TQQQ.OQ : {'BID': 35.18, 'ASK': 35.2}
18:40:56.117652 - Update received for URTY.P : {'BID': 55.33, 'ASK': 55.45}
18:40:56.120651 - Update received for UWM.P : {'BID': 38.48, '

,Instrument,BID,ASK
0,JAEREN.OL,<NA>,<NA>
1,LEAD.KL,0.555,0.61
2,ROANTA.BX,<NA>,<NA>
3,BPN.MX,620.0,699.0
4,BRIL.TA,3353.0,3400.0
5,TKYVY.PK,<NA>,<NA>
6,066130.KQ,<NA>,<NA>
7,UNLB.BO,<NA>,<NA>
8,MGNT.MM,<NA>,<NA>
9,LNTAq.L,<NA>,<NA>


18:40:59.103350 - Status received for ZSJ.F : {'ID': 53787, 'Type': 'Status', 'Key': {'Service': 'ELEKTRON_DD', 'Name': 'ZSJ.F'}, 'State': {'Stream': 'Closed', 'Data': 'Suspect', 'Code': 'NotFound', 'Text': '**The record could not be found'}}
Stream # 4
18:40:59.296316 - Update received for ZG.OQ : {'BID': 41.36, 'ASK': 41.69}
18:40:59.477317 - Update received for AIA.NZ : {'ASK': 7.64}
18:41:00.312102 - Update received for TNA.P : {'BID': 46.33, 'ASK': 46.35}
18:41:00.312102 - Update received for IVOO.P : {'BID': 172.36, 'ASK': 172.89}
18:41:00.315102 - Update received for VTWO.OQ : {'BID': 76.14, 'ASK': 76.35}
18:41:00.318144 - Update received for URTY.P : {'BID': 55.29, 'ASK': 55.51}
18:41:00.321653 - Update received for UWM.P : {'BID': 38.47, 'ASK': 38.57}
18:41:00.325678 - Update received for DIA.P : {'BID': 333.0, 'ASK': 333.1}
18:41:01.170984 - Update received for IVV.P : {'BID': 419.74, 'ASK': 419.85}
18:41:01.170984 - Update received for VOO.P : {'BID': 384.15, 'ASK': 384.25}


### Close Streams Sequentially (at any time)
Once all streams are closed- all the updates should cease

In [50]:
for i in range(len(streams_list)):
    streams_list[i].close()
    print("Stream closed #", i)

18:41:19.572079 - Update received for TQQQ.OQ : {'BID': 35.18, 'ASK': 35.2}
Stream closed # 0
18:41:20.600044 - Update received for IVOO.P : {'BID': 171.28, 'ASK': 172.89}
18:41:20.600044 - Update received for URTY.P : {'BID': 55.33, 'ASK': 55.5}
18:41:20.600044 - Update received for UWM.P : {'BID': 38.49, 'ASK': 38.56}
18:41:20.600044 - Update received for MDY.P : {'BID': 465.75, 'ASK': 466.46}
18:41:20.600044 - Update received for IVV.P : {'BID': 419.75, 'ASK': 419.85}
18:41:20.600044 - Update received for VOO.P : {'BID': 384.16, 'ASK': 384.24}
18:41:20.601044 - Update received for UPRO.P : {'BID': 48.07, 'ASK': 48.11}
18:41:20.601044 - Update received for DIA.P : {'BID': 333.0, 'ASK': 333.08}
18:41:20.601044 - Update received for TQQQ.OQ : {'BID': 35.17, 'ASK': 35.19}
18:41:20.602043 - Update received for TQQQ.OQ : {'BID': 35.17, 'ASK': 35.18}
18:41:20.602043 - Update received for TQQQ.OQ : {'BID': 35.17, 'ASK': 35.19}
Stream closed # 1
18:41:21.456042 - Update received for UWM.P : 

In [52]:
rd.close_session()